# Image processing
by Shadi Alameddin <alameddin@mib.uni-stuttgart.de>, January 2022

additional material for the course _Data Processing for Engineers and Scientists_ at the University of Stuttgart

## Content

### Part 1: image manipulation
- How to read and write images
- Manipulate individual pixels of an image
- Convert RGB image to a grayscale one

### Part 2: image segmentation
- Non-binary segmentation
- Smoothing filters
- Binary segmentation

### Part 3: binary image manipulation
- Morphological operations
- Circle detection algorithm
- Extract sttistical infromation from an image
- Extra: segmentation and crack detection

### Optional part:
- Improve the output of the circle detection algorithm
- Segment the full image instead of the sample we used here
- Use more than one morphological operation to achieve a better result
- Try to isolate the cracks in the particles
- Elaborate on the provided examples to build a basic __traffic-sign recognition system__

## Part 1: loading and manipulating images
__given__: an RGB image file with pixels in $$ \mathbb{A} = \{0,\cdots,255\}^3$$
- load it into Python (use `matplotlib` and `cv2`: OpenCV)
- visualise it
- delete some rows or columns of the image
- compress it to a grayscale image

In [ ]:
# pip install opencv-python
import numpy as np
import cv2 as cv 
from matplotlib import rc
import matplotlib.pyplot as plt

rc('font',**{'family':'sans-serif','sans-serif':['DejaVu Sans'], 'size':16})
plt.rcParams['xtick.bottom'] = plt.rcParams['xtick.labelbottom'] = False
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True
plt.rcParams['axes.grid']=True


In [ ]:
# read an image in RGB format
original_image_rgb = plt.imread('micrograph.tif')

fig, ax= plt.subplots(1,3,figsize=[16,4])
ax[0].imshow(original_image_rgb)
ax[0].set_title('Original RGB image')

# recall that (0,0) is on the top-left
corrupted_image = np.copy(original_image_rgb)
corrupted_image[750:800,:,:]=0
corrupted_image[:,2000:2050,:]=[255,0,0]
ax[1].imshow(corrupted_image)
ax[1].set_title('Manipulated image')

original_image_gray = cv.cvtColor(original_image_rgb,cv.COLOR_RGB2GRAY)
ax[2].imshow(original_image_gray,cmap="gray")
ax[2].set_title('Grayscale representation')

print(f'matrix shape of the original image{original_image_rgb.shape}')
print(f'imported image has {original_image_rgb.shape[0]} pixels vertically and {original_image_rgb.shape[1]} horizontally')
print(f'minumum and maximum values in the original matrix: {original_image_rgb.min(),original_image_rgb.max()}')
print(f'matrix shape of the grayscale image {original_image_gray.shape}')
print(f'minumum and maximum values in the grayscale matrix: {original_image_gray.min(),original_image_gray.max()}')

## Part 2: image segmentation
- crop a small "_representative_" sample of the grayscale image
- look at the values of each pixel
- set a "_reasonable_" threshold -> first segmented version (non-binary segmentation)

In [ ]:
yrange = np.arange(50,350)[:,None]
xrange = np.arange(50,350)[None,:]
image=original_image_rgb[yrange,xrange,:]
image_gray=original_image_gray[yrange,xrange]

fig, ax= plt.subplots(1,3,figsize=[16,4])
ax[0].set_title('Sample to be investigated')
ax[0].imshow(image_gray,cmap="gray")

ax[1].set_title('Pixel intensity histogram')
ax[1].hist(image_gray.flatten())
ax[1].set_box_aspect(1)
ax[1].set_xlabel('gray level')
ax[1].set_ylabel('number of pixels')

# TODO: look at cv.THRESH_BINARY
segmented_image_gray = np.copy(image_gray)
rng = segmented_image_gray<=175
segmented_image_gray[rng]=0
ax[2].set_title('Non-binary segmented image')
ax[2].imshow(segmented_image_gray,cmap="gray")

- smooth the image
- convert the image to a binary segmented one

In [ ]:
kernel_size = 5
# segmented_image_gray = cv.GaussianBlur(image,(kernel_size,kernel_size),0)
smoothed_image = cv.medianBlur(image,kernel_size)
fig, ax= plt.subplots(1,3,figsize=[16,4])
ax[0].set_title('Smoothed image')
ax[0].imshow(smoothed_image,cmap="gray")

segmented_image_gray = cv.cvtColor(smoothed_image,cv.COLOR_RGB2GRAY)
segmented_image_gray[segmented_image_gray<=185]=0
segmented_image_gray[segmented_image_gray>=195]=255
ax[1].set_title('Non-binary segmentation')
ax[1].imshow(segmented_image_gray,cmap="gray")

binary_image = segmented_image_gray
rng=segmented_image_gray>=186
segmented_image_gray[rng]= 1
segmented_image_gray[~rng] = 0
ax[2].set_title('Binary segmentation')
ax[2].imshow(binary_image,cmap="gray")

- check color values in each channel

In [ ]:
fig, ax= plt.subplots(1,3,figsize=[16,4])
ax[0].hist(image[...,0].flatten(),color='red')
ax[0].set_title('Red intensity histogram')
ax[0].set_box_aspect(1)
ax[0].set_xlabel('gray level')
ax[0].set_ylabel('number of pixels')

ax[1].hist(image[...,1].flatten(),color='green')
ax[1].set_title('Green intensity histogram')
ax[1].set_box_aspect(1)
ax[1].set_xlabel('gray level')
ax[1].set_ylabel('number of pixels')

ax[2].hist(image[...,2].flatten(),color='blue')
ax[2].set_title('Blue intensity histogram')
ax[2].set_box_aspect(1)
ax[2].set_xlabel('gray level')
ax[2].set_ylabel('number of pixels')

- enhance image contrast
- what about making only one dominant color in each pixel?

In [ ]:
modified_image = np.copy(image)
modified_image[modified_image<=175] = 0
fig, ax= plt.subplots(1,3,figsize=[16,4])
ax[0].set_title('Image without redundant info')
ax[0].imshow(modified_image)

flatten_image = image.reshape(-1,3)
flatten_color_segmented_image = np.zeros_like(flatten_image)
idx = np.argmax(flatten_image,axis=1)
flatten_color_segmented_image[idx==0]=[255,0,0]
flatten_color_segmented_image[idx==1]=[0,255,0]
flatten_color_segmented_image[idx==2]=[0,0,255]
color_segmented_image = flatten_color_segmented_image.reshape(image.shape)
ax[1].set_title('Dominant color segmentation')
ax[1].imshow(color_segmented_image)

binary_image = np.copy(flatten_color_segmented_image)
binary_image[idx==2]=0
binary_image[idx!=2]=1
binary_image = np.mean(binary_image.reshape(image.shape),axis=2)
ax[2].imshow(binary_image,cmap="gray")

- filtering accounts for nonlocal information $\blacktriangleright$ thresholding of the filtered image

In [ ]:
fig, ax= plt.subplots(1,3,figsize=[16,4])
ax[0].set_title('Image without redundant info')
ax[0].imshow(modified_image)

kernel_size = 17
color_segmented_image2 = cv.medianBlur(color_segmented_image,kernel_size)
ax[1].set_title('Dominant color segmentation')
ax[1].imshow(color_segmented_image2)

binary_image = cv.cvtColor(color_segmented_image2,cv.COLOR_RGB2GRAY)
rng = (binary_image>15) & (binary_image<30)
binary_image[rng]=0
binary_image[~rng]=1
ax[2].set_title('Segmented image')
ax[2].imshow(binary_image,cmap="gray")

- It is also possible to use region labeling to get obtain a segmented version of the provided image

## Part 3: binary image manipulation

Morphological operations:
- Dilation - grow image regions [set pixel to max (white) within a box]
- Erosion - shrink image regions [set pixel to min (black) within a box]
- Opening - structured removal of image region boundary pixels [erosion followed by a dilation, remove white dots]
- Closing - structured filling in of image region boundary pixels [a dilation followed by an erosion, remove black dots]

In [ ]:
binary_image2 = np.bitwise_not(binary_image)
# kernel = np.ones((3, 3), np.uint8)/9
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(5,5))

itr = 4
image_dialated = cv.morphologyEx(binary_image2, cv.MORPH_DILATE, kernel, iterations=itr)
image_eroded = cv.morphologyEx(binary_image2, cv.MORPH_ERODE, kernel, iterations=itr)
image_opening = cv.morphologyEx(binary_image2, cv.MORPH_OPEN, kernel, iterations=itr)
image_closing = cv.morphologyEx(binary_image2, cv.MORPH_CLOSE, kernel, iterations=itr)

fig, ax= plt.subplots(2,2,figsize=[12,12])
ax[0,0].set_title('Dialated image')
ax[0,0].imshow(image_dialated)
ax[0,1].set_title('Eroded image')
ax[0,1].imshow(image_eroded)
ax[1,0].set_title('Closed image')
ax[1,0].imshow(image_closing)
ax[1,1].set_title('Opened image')
ax[1,1].imshow(image_opening)

> <font color='red'>**NOTE**</font>
> * Iterative application of opening and closing multiple times with the same kernel should be identical to applying them once. In other words, the morphological opening and closing are idempotent [Statistical Analysis of Microstructures in Materials Science by Ohser and Mücklich]. However, we see a different result using OpenCV "morphologyEx" function.
> * If you execute "help(cv.morphologyEx)" you'll find the following comment:
<font color='gray'>
@note The number of iterations is the number of times erosion or dilatation operation will be applied. For instance, an opening operation (#MORPH_OPEN) with two iterations is equivalent to apply successively: erode -> erode -> dilate -> dilate (and not erode -> dilate -> erode -> dilate).
</font>
> * Opening and closing operations can also be tested with:
```open(open(img,k),k) == open(img,k)?``` and ```close(close(img,k),k) == close(img,k)?```.
Both should be satisfied unless the implmentation doesn't try to replicate the theory.

- Circle detection algorithm

In [ ]:
img_255 = np.array(image_eroded,dtype=np.uint8)
synthetic_image = np.zeros_like(image_eroded)

circles = cv.HoughCircles(img_255,cv.HOUGH_GRADIENT,1.0,50,param1=1e-6,param2=12,minRadius=3,maxRadius=80)
circles = np.uint16(circles)

for i in circles[0,:]:
    synthetic_image = cv.circle(synthetic_image, (i[0],i[1]), i[2], 1, -1)

fig, ax= plt.subplots(1,2,figsize=[12,8])
ax[0].set_title('Input binary image')
ax[0].imshow(img_255)
ax[1].set_title('Synthetic image')
ax[1].imshow(synthetic_image)

- Statistical information from an image

In [ ]:
from scipy.signal import savgol_filter

def extract_plot_statistics(h_stripe,v_stripe,sg_window,sg_order):
    h_vol_av=[]
    v_vol_av=[]
    reinforcement_depth = []

    for i in np.arange(0,synthetic_image.shape[0] - h_stripe):
        id_range = range(i, i + h_stripe)
        h_vol_av.append( np.count_nonzero(synthetic_image[id_range,:]) / synthetic_image[id_range,:].size)

    for i in np.arange(0,synthetic_image.shape[1] - v_stripe):
        id_range = range(i, i + v_stripe)
        v_vol_av.append( np.count_nonzero(synthetic_image[:,id_range]) / synthetic_image[:,id_range].size)

        reinforcement_depth.append(np.nonzero(np.sum(synthetic_image[:,id_range],axis=1))[-1][-1] if np.sum(synthetic_image[:,id_range]) > 0 else 0)
   
    smotthed_h_vol_av = savgol_filter(h_vol_av, sg_window, sg_order)
    smotthed_v_vol_av = savgol_filter(v_vol_av, sg_window, sg_order)
    smoothed_reinforcement_depth = savgol_filter(reinforcement_depth, sg_window, sg_order)

    smotthed_h_vol_av[smotthed_h_vol_av<=0] = 0
    smotthed_v_vol_av[smotthed_v_vol_av<0] = 0
    smoothed_reinforcement_depth[smoothed_reinforcement_depth<0] = 0

    fig, ax= plt.subplots(2,3,figsize=[16,10])
    ax[0,0].set_visible(False)
    ax[0,2].set_visible(False)

    ax[1,0].plot(h_vol_av,range(len(h_vol_av)),'k', linestyle='--',linewidth=2)
    ax[1,0].plot(smotthed_h_vol_av,range(len(h_vol_av)),'k',linewidth=3)
    ax[1,0].invert_xaxis()
    ax[1,0].invert_yaxis()
    ax[1,0].set_box_aspect(1)
    # ax[1,0].set_title('vol frac per row (depth)')
    ax[1,0].set_xlabel('volume fraction w.r.t. depth[-]')
    ax[1,0].xaxis.set_label_position("top")
    
    ax[0,1].plot(range(len(v_vol_av)),v_vol_av,'k', linestyle='--',linewidth=2)
    ax[0,1].plot(range(len(v_vol_av)),smotthed_v_vol_av,'k',linewidth=3)
    ax[0,1].set_title('vol frac per column')
    ax[0,1].set_ylabel('volume fraction [-]')
    ax[0,1].set_box_aspect(1)

    ax[1,1].imshow(synthetic_image)
    ax[1,1].set_xticks([])
    ax[1,1].set_yticks([])

    ax[1,2].plot(reinforcement_depth,'k', linestyle='--',linewidth=2)
    ax[1,2].plot(smoothed_reinforcement_depth,'k',linewidth=3)
    ax[1,2].set_title('reinforcement depth')
    ax[1,2].set_ylabel('depth [pixels]')
    ax[1,2].invert_yaxis()
    ax[1,2].set_box_aspect(1)
    ax[1,2].yaxis.tick_right()
    ax[1,2].yaxis.set_label_position("right")

In [ ]:
from ipywidgets import interact, widgets

interact(extract_plot_statistics,h_stripe=widgets.IntSlider(min=1, max=29, step=1, value=1),
         v_stripe=widgets.IntSlider(min=1, max=29, step=1, value=1),
         sg_window=widgets.IntSlider(min=3, max=50, step=2, value=3),
         sg_order=widgets.IntSlider(min=1, max=5, step=1, value=1))

### Extra: segmentation and crack detection

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

# get image and read properties

# rgb_image = plt.imread('micrograph.tif')[:800,:800]
rgb_image = plt.imread('micrograph.tif')[:1750,:2200]

print('properties of original image: ')
print(f'matrix_shape {rgb_image.shape}')
print(f'resolution {rgb_image.shape[0]} x {rgb_image.shape[1]}')

# contrast improvement by enhancing dark pixels from the background
gray_image = cv.cvtColor(rgb_image, cv.COLOR_BGR2GRAY)
gray_image[gray_image<180]=0

# Image blurring -> image smoothing & noise removal
blured_image = cv.medianBlur(rgb_image, 9)
# blured_image = cv.bilateralFilter(rgb_image, 5,50,50)


fig, ax= plt.subplots(1,3,figsize=[16,4])    
ax[0].imshow(rgb_image)
ax[1].imshow(gray_image,cmap='gray')
ax[2].imshow(blured_image)

In [ ]:
modified_image = np.copy(rgb_image)
modified_image[modified_image<=175] = 0
fig, ax= plt.subplots(1,3,figsize=[16,4])
ax[0].set_title('Image without redundant info')
ax[0].imshow(modified_image)

flatten_image = rgb_image.reshape(-1,3)
flatten_color_segmented_image = np.zeros_like(flatten_image)
idx = np.argmax(flatten_image,axis=1)
flatten_color_segmented_image[idx==0]=[255,0,0]
flatten_color_segmented_image[idx==1]=[0,255,0]
flatten_color_segmented_image[idx==2]=[0,0,255]
color_segmented_image = flatten_color_segmented_image.reshape(rgb_image.shape)

kernel_size = 17
color_segmented_image = cv.medianBlur(color_segmented_image,kernel_size)
ax[1].set_title('Dominant color segmentation')
ax[1].imshow(color_segmented_image)

binary_image = cv.cvtColor(color_segmented_image,cv.COLOR_RGB2GRAY)
rng = (binary_image>15) & (binary_image<30)
binary_image[rng]=1
binary_image[~rng]=0
ax[2].set_title('Segmented image')
ax[2].imshow(binary_image,cmap="gray")

In [ ]:
fig, ax= plt.subplots(1,3,figsize=[16,4])
ax[0].imshow(gray_image,cmap="gray")
ax[1].imshow(binary_image,cmap="gray")
ax[2].imshow(binary_image * gray_image,cmap="gray")

In [ ]:
fig, ax= plt.subplots(1,3,figsize=[16,4])

kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(5,5))
binary_image_eroded = cv.morphologyEx(binary_image, cv.MORPH_ERODE, kernel, iterations=8)
ax[0].imshow(binary_image_eroded,cmap="gray")


laplacian = cv.Laplacian(gray_image,cv.CV_16S,ksize=9)
ax[1].imshow(laplacian)

cracks_image=laplacian*binary_image_eroded
ax[2].imshow(cracks_image,cmap="gray")

rng = (cracks_image<15e3) 
cracks_image[rng]=255
cracks_image[~rng]=0
ax[2].imshow(cracks_image,cmap="gray")

In [ ]:
fig, ax= plt.subplots(1,3,figsize=[16,4])


n=5
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(n,n))

dialated = cv.morphologyEx(cracks_image, cv.MORPH_ERODE, kernel, iterations=1)
ax[0].imshow(dialated,cmap="gray")

opened = cv.morphologyEx(dialated, cv.MORPH_OPEN, kernel, iterations=2)
ax[1].imshow(opened,cmap="gray")

n=3
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(n,n))
closed = cv.morphologyEx(opened, cv.MORPH_CLOSE, kernel, iterations=3)
ax[2].imshow(closed,cmap="gray")

In [ ]:
fig, ax= plt.subplots(1,2,figsize=[18,6])
ax[0].imshow(rgb_image)

rgb_cracks = np.copy(rgb_image)
rgb_cracks[closed[:,:]==0]=[255,0,0]
ax[1].imshow(rgb_cracks)